### Create wind and timepoint input for SWAN

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
filepath = r'C:\Users\Mariliasr\OneDrive\Documentos\_UFRGS\Mestrado\Dissert\datasets\era5\wind'
wind_nc = 'wind_era5_2005.nc'
grid_resolution = 0.25
u10 = 'u10'
v10 = 'v10'

x_output_filename = 'x_wind_2005.wnd'
y_output_filename = 'y_wind_2005.wnd'

delft_path = r'C:\Users\Mariliasr\OneDrive\Documentos\_UFRGS\Mestrado\Dissert\hindcast\2015' 
timepoint_old_filename = 'timepoints_old.txt'
timepoint_filename = 'timepoins.csv'

In [25]:
%cd $filepath

C:\Users\Mariliasr\OneDrive\Documentos\_UFRGS\Mestrado\Dissert\datasets\era5\wind


In [26]:
wind_xr = xr.open_dataset(wind_nc)

In [27]:
wind_xr.var

<bound method ImplementsDatasetReduce._reduce_method.<locals>.wrapped_func of <xarray.Dataset>
Dimensions:    (latitude: 121, longitude: 201, time: 2160)
Coordinates:
  * longitude  (longitude) float32 -70.0 -69.75 -69.5 ... -20.5 -20.25 -20.0
  * latitude   (latitude) float32 -20.0 -20.25 -20.5 ... -49.5 -49.75 -50.0
  * time       (time) datetime64[ns] 2005-01-01 ... 2005-03-31T23:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2019-11-06 15:23:44 GMT by grib_to_netcdf-2.14.0: /opt/ecmw...>

In [28]:
wind_xr = wind_xr.sel(latitude=slice(-24, -37.5), longitude=slice(-56.5, -45))

In [29]:
wind_xr.var

<bound method ImplementsDatasetReduce._reduce_method.<locals>.wrapped_func of <xarray.Dataset>
Dimensions:    (latitude: 55, longitude: 47, time: 2160)
Coordinates:
  * longitude  (longitude) float32 -56.5 -56.25 -56.0 ... -45.5 -45.25 -45.0
  * latitude   (latitude) float32 -24.0 -24.25 -24.5 ... -37.0 -37.25 -37.5
  * time       (time) datetime64[ns] 2005-01-01 ... 2005-03-31T23:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2019-11-06 15:23:44 GMT by grib_to_netcdf-2.14.0: /opt/ecmw...>

In [30]:
#Separate variables from nc file

longitude = wind_xr['longitude'].values
latitude = wind_xr['latitude'].values
time = wind_xr['time'].values
u_wind = wind_xr[u10].values
v_wind = wind_xr[v10].values

In [31]:
latitude.min()

-37.5

In [32]:
# Create hourly timepoint vector of strings

time = pd.to_datetime(time)
timepoints = []
delta = time[-1] - time[0]
minutes = delta.days*24*60 + delta.seconds/60 + 60
timepoint_np = np.arange(0, minutes, 60)

for timepoint in range(len(timepoint_np)):
    timepoint_str = 'TIME = {:.5e} minutes since {:%Y-%m-%d %H:%M:%S} +00:00'.format(
        timepoint_np[timepoint], time[0])
    timepoints.append(timepoint_str)
    
len(timepoints) == len(time)

True

In [33]:
%cd $delft_path

C:\Users\Mariliasr\OneDrive\Documentos\_UFRGS\Mestrado\Dissert\hindcast\2015


In [34]:
# Create timepoints old version

fin = open(timepoint_old_filename, 'w')
for timepoint in range(len(timepoint_np)):
    fin.write('''
[Timepoint]
   Time                 =  {:.7e}
   WaterLevel           =  0.0000000e+00
   XVeloc               =  0.0000000e+00        
   YVeloc               =  0.0000000e+00'''.format(timepoint_np[timepoint]))

fin.close()

In [23]:
# Create timepoints new version
fin = open(timepoint_filename, 'w')
for timepoint in range(len(time)):
    fin.write(f'''
{time[timepoint]}''')

fin.close()

In [38]:
%cd $filepath

C:\Users\Mariliasr\OneDrive\Documentos\_UFRGS\Mestrado\Dissert\datasets\era5\wind


In [42]:
# Create u .wnd file

fin = open(x_output_filename, 'w')
fin.write(
f'''FileVersion = 1.03
filetype = meteo_on_equidistant_grid
NODATA_value = -999
grid_unit = degree
n_cols = {len(longitude)-1}
n_rows = {len(latitude)-1}
x_llcorner = {longitude.min()} 
y_llcorner = {latitude.min()}
dx = {grid_resolution}
dy = {grid_resolution}
n_quantity = 1
quantity1 = x_wind
unit1 = m s-1''')

fin.close()

fin=open(x_output_filename, 'a+')
for timepoint in range(len(timepoints)):
    fin.write(
'''{}
{}'''.format(timepoints[timepoint],
             pd.DataFrame(u_wind[timepoint][:][:]).to_csv(index=False,
                                                          header=False,
                                                          sep=',',
                                                          float_format='%10.5f',
                                                          na_rep= -999)))
fin = open(x_output_filename, "rt")
data = fin.read()
data = data.replace(',', '')
fin.close()
    
fin = open(x_output_filename, "wt")
fin.write(data)
fin.close()

In [43]:
# Create v .wnd file

fin = open(y_output_filename, 'w')
fin.write(
f'''FileVersion = 1.03
filetype = meteo_on_equidistant_grid
NODATA_value = -999
grid_unit = degree
n_cols = {len(longitude)-1}
n_rows = {len(latitude)-1}
x_llcorner = {longitude.min()} 
y_llcorner = {latitude.min()}
dx = {grid_resolution}
dy = {grid_resolution}
n_quantity = 1
quantity1 = y_wind
unit1 = m s-1''')

fin.close()

fin=open(y_output_filename, 'a+')
for timepoint in range(len(timepoints)):
    fin.write(
'''{}
{}'''.format(timepoints[timepoint],
             pd.DataFrame(v_wind[timepoint][:][:]).to_csv(index=False,
                                                          header=False,
                                                          sep=',',
                                                          float_format='%10.5f',
                                                          na_rep= -999)))
fin = open(y_output_filename, "rt")
data = fin.read()
data = data.replace(',', '')
fin.close()
    
fin = open(y_output_filename, "wt")
fin.write(data)
fin.close()